# Prueba de código: Ingeniero de Datos - Orenes Core
Esta prueba está pensada para realizar la entrevista del puesto de Ingeniero de Datos. El objetivo es que este cuaderno sirva de hilo conductor en la entrevista técnica, la prueba está pensada para que puedas mostrar lo que sabes, no para descartar o acreditar candidatos. Es una herramienta a tu disposición, úsala de la forma que creas que mejor muestra tu conocimiento o tus cualidades. Por ejemplo: Si no puedes resolver un paso puedes simular la solución para mostrar otra cosa, si piensas que hay varias formas de resolver un problema o si piensas que hay algún añadido que no se pide siéntete libre de incluirlo.


La entrega consistirá en una copia de esta plantilla con los apartados completados y se revisará durante la entrevista. Esta plantilla no puede editarse, **ANTES DE EMPEZAR** puedes realizar una copia editable en Archivo/Guardar una Copia en Drive. Se valorará durante la entrevista si has llevado un control de versiones en git (usando GitHub por ejemplo).

## Entorno Spark
  Esta pieza de código te permitirá usar una interfaz de Spark en Colab para resolver tus ejercicios.

In [94]:
# Install pyspark
#!pip install pyspark
# Import SparkSession
from pyspark.sql import SparkSession

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark
# Import a Spark function from library
from pyspark.sql.functions import col, max, mean

In [95]:
import time

## Paso 1
Carga los datos del fichero housing_train.csv que guardado en la carpeta sample_data en un data frame de Spark.

Se hace uso del argumento "header", ya que los datos vienen con cabeceras, además se usa "inferSchema" ya que permite que a cada columna, se le asigne el mejor tipo de dato.

In [96]:
dataset_inicial = spark.read.csv("sample_data/housing_train.csv", header=True, inferSchema=True)
dataset_inicial.columns
dataset_inicial.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
+---------+--------+----

## Paso 2
Crea otro data frame a apartir de un muestreo aleatorio del 20% del data frame creado. Después crea otro data frames con el resto de registros. Comprueba que al unir los dos data frames creados no existen pérdidas ni duplicados de registors respecto al obtenido en el paso anterior.

Se extrae el 20% de los datos haciendo uso de la función "[sample](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.sample.html)", que junto a su argumento "fraction" permite escoger el porcentaje de datos que se quiere extraer de forma ***aleatoria***, sin embargo para poder reproducir el ejercicio de forma exacta, se añade el argumento "seed".

In [97]:
muestra_df = dataset_inicial.sample(fraction=0.2, seed=1)

Para obtener un dataframe con el resto de datos, se hace usode la función "[exceptAll](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.exceptAll.html)".

In [98]:
resto_df = dataset_inicial.exceptAll(muestra_df)

A continuación se unen ambos df, haciendo uso de la función "[union](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html)", la cual permite realizar un merge entre dos dataframe.

In [99]:
muestra_and_resto = muestra_df.union(resto_df)

Finalmente, para comprobar que no se ha producido ninguna pérdida durante el proceso, se comprueba la longitud tanto del dataframe inicial, como el resultado tras unir ambos dataframe. Además puede darse el caso de que durante el proceso se duplique una fila y se pierda otra, nos encontrariamos en el caso en el que la primera comprobación indicaría que todo esta bien, ya que tienen el mismo número de fila, pero esto puede deverse a que tenemos la duplicada. Es por ello que con la función "[distinct](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.distinct.html)", evitamos que esto ocurra.

In [100]:
if muestra_and_resto.count() == dataset_inicial.count() and muestra_and_resto.distinct().count() == dataset_inicial.distinct().count():
    print("No hay pérdidas ni duplicados al unir los DataFrames")
else:
    print("Se han perdido o duplicado registros al unir los DataFrames")

No hay pérdidas ni duplicados al unir los DataFrames


## Paso 3
Realiza un histograma de cada uno de los campos y compara los dos data frames creados en el paso anterior.

## Paso 4
Partiendo del data frame del primer paso. Obtén el registro que más al norte esté según el número de habitaciones que tenga. Es decir, el registro más al norte entre todos los registros con una sola habitación, el más al norte entre todos los registros de dos habitaciones, y así sucesivamente.

Se obtiene el número máximo de habitación en el dataframe. A continuación, por cada número de habitaciones, se coge del dataframe inicial, todas las filas que tenga el mismo número de habitaciones y de ellas, se escoge la habitación más al norte, es decir, ordenando de manera descendiente y cogiendo la primera fila.

In [101]:
def select_records_by_num_bedrooms_v1(df):
    max_bedrooms = df.select(max("total_bedrooms")).collect()[0][0]
    selected_records = []
    # Se recorren todos los número de habitaciones que hay
    for num_bedrooms in range(1, int(max_bedrooms) + 1):
        # Se seleccionan los registros con el número de habitaciones actual
        records = df.filter(col("total_bedrooms") == num_bedrooms)
        selected_record = records.orderBy("latitude", ascending=False).limit(1).collect()
        # Se comprueba que exista un registro para dicho número de abitación
        if selected_record:
            selected_record = selected_record[0]
            # Añadir el registro seleccionado a la lista
            selected_records.append(selected_record)
    return spark.createDataFrame(selected_records)

La opción anterior, tarda demasiado tiempo, ya que se hace un uso de un for, por lo que hay que ir iterando, sin embargo es más legible entender el código. Otra opción sería:
    - Agrupo por "total_bedrroms" (groupBy)
    - De cada agrupación, se coge la fila de mayor valor de latitude (max)

De esta manera haciendo uso de "groupBy" y de "agg", todo puede ser más rápido.

In [106]:
def select_records_by_num_bedrooms_v2(df):
    # Se agrupa por número de habitaciones y se obtiene el registro más al norte para cada grupo
    max_latitude_by_rooms = (dataset_inicial
                             .groupBy("total_bedrooms")
                             .agg(max("latitude").alias("max_latitude")))
    # Se une el DataFrame original con los registros más al norte para cada grupo
    joined_df = dataset_inicial.join(max_latitude_by_rooms, "total_bedrooms")
    # Se selecciona solo las columnas deseadas y ordenar por número de habitaciones
    result_df = (joined_df
                 .select("longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population",
                         "households", "median_income", "median_house_value")
                 .orderBy("total_bedrooms", ascendent=False))
    return result_df

**Comprobación del tiempo que tardan**

Iterando con un for.

In [103]:
# Se mide el tiempo de ejecución
star_time_v1 = time.time()
dataset_resultado = select_records_by_num_bedrooms_v1(dataset_inicial)
elapsed_time_v1 = time.time() - star_time_v1
print("Tiempo de ejecución de la función utilizando for: ", elapsed_time_v1)

# Se muestra el resultado
dataset_resultado.columns
dataset_resultado.show()

Tiempo de ejecución de la función utilizando for:  350.05750012397766
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|   -122.5|   37.79|              52.0|        8.0|           1.0|      13.0|       1.0|      15.0001|          500001.0|
|  -117.79|   35.21|               4.0|        2.0|           2.0|       6.0|       2.0|        2.375|          137500.0|
|  -122.37|    37.6|              26.0|       15.0|           3.0|      11.0|       3.0|        5.048|          350000.0|
|  -121.47|   38.51|              52.0|       20.0|           4.0|      74.0|       9.0|        3.625|           80000.0|
|  -121.96|   37.13|              26.0|       50.0|           5.0|      17.0

Utilizando "groupBy" y "agg"

In [107]:
# Se mide el tiempo de ejecución
star_time_v2 = time.time()
dataset_resultado_v2 = select_records_by_num_bedrooms_v2(dataset_inicial)
elapsed_time_v2 = time.time() - star_time_v2
print("Tiempo de ejecución de la función optimizada: ", elapsed_time_v2)

# Se muestra el resultado
dataset_resultado_v2.columns
dataset_resultado_v2.show()

Tiempo de ejecución de la función optimizada:  0.022614717483520508
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|   -122.5|   37.79|              52.0|        8.0|           1.0|      13.0|       1.0|      15.0001|          500001.0|
|  -117.79|   35.21|               4.0|        2.0|           2.0|       6.0|       2.0|        2.375|          137500.0|
|  -117.27|   34.17|              16.0|       30.0|           3.0|      49.0|       8.0|        4.625|          250000.0|
|  -122.37|    37.6|              26.0|       15.0|           3.0|      11.0|       3.0|        5.048|          350000.0|
|  -117.76|   35.22|               4.0|       18.0|           3.0|       8.0| 

## Paso 5
Partiendo de los dos data frames del Paso 2. Calcula la media de las latitudes de los cada data frame agregado por el el número de dormitorios. Calcula las difrencias entre las medias de cada data frame para el mismo número de dormitorios. Es decir, la diferencia entre la latitud media de los registros del data frame del 20% con un solo dormitorio y la latitud media del data frame del 80% con un solo dormitorio; Realiza esta operación para todo número de habitaciones.

Se comienza, agrupando por la columna "total_bedrooms", seguidamente con la función "[agg](https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.DataFrame.agg.html)", se calcula la media de cada grupo y finalmente se se nombra la columna.

In [112]:
media_muestra = muestra_df.groupBy("total_bedrooms").agg(mean("latitude").alias("mean_latitude_muestra")).orderBy("total_bedrooms", ascendent=False)
print("Resultado dataset 20%")
media_muestra.show()

media_resto = resto_df.groupBy("total_bedrooms").agg(mean("latitude").alias("mean_latitude_resto")).orderBy("total_bedrooms", ascendent=False)
print("Resultado dataset resto")
media_resto.show()

Resultado dataset 20%
+--------------+---------------------+
|total_bedrooms|mean_latitude_muestra|
+--------------+---------------------+
|           3.0|                 37.6|
|           4.0|               37.625|
|           5.0|                32.75|
|           9.0|                34.02|
|          10.0|                36.06|
|          11.0|    34.48333333333333|
|          12.0|               35.675|
|          13.0|                33.12|
|          14.0|                 33.8|
|          15.0|                37.98|
|          16.0|                35.24|
|          17.0|                35.32|
|          18.0|   36.449999999999996|
|          19.0|                33.83|
|          20.0|                35.78|
|          21.0|                38.59|
|          22.0|               34.045|
|          23.0|                33.96|
|          25.0|                33.28|
|          28.0|               35.715|
+--------------+---------------------+
only showing top 20 rows

Resultado datase

Una vez que ya tenemos cada tabla con su respectiva media para cada número de habitación, se realizan los siguientes pasos:
    - Se unen ambos dataframe, mediante la columna total_bedrooms, para así tener el número de habitación con sus correspondientes medias de cada dataframe.
    - Se crea la columna "diferencia", compuesta por la resta de cada una de las medias.
    - Finalmente se ordena de en función de la columna habitaciones.

In [115]:
diferencias = media_muestra.join(media_resto, "total_bedrooms") \
    .withColumn("diferencia", col("mean_latitude_muestra") - col("mean_latitude_resto")) \
    .orderBy("diferencia", ascendent=False) \
    .select("total_bedrooms", "diferencia")
diferencias.show()

+--------------+-------------------+
|total_bedrooms|         diferencia|
+--------------+-------------------+
|        1856.0| -4.979999999999997|
|        1655.0| -4.899999999999999|
|        1324.0| -4.899999999999999|
|        1039.0| -4.630000000000003|
|        1542.0| -4.580000000000005|
|        1158.0| -4.479999999999997|
|          19.0| -4.240000000000002|
|          31.0| -4.229999999999997|
|        1893.0| -4.130000000000003|
|         858.0| -4.048333333333339|
|        1110.0| -4.035000000000004|
|        1213.0|-3.9299999999999997|
|         992.0|-3.9279999999999973|
|        1914.0|-3.8900000000000006|
|         714.0| -3.865000000000002|
|         117.0|-3.7674999999999983|
|        1575.0| -3.719999999999999|
|        1050.0|-3.6099999999999994|
|         870.0| -3.604999999999997|
|         907.0| -3.520000000000003|
+--------------+-------------------+
only showing top 20 rows



## Paso 6
Calcula cualquier medida que consideres interesante en utilizando Spark SQL. Por ejemplo, la media de dormitorios agrupado por las habitaciones que tiene cada registro.